## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [2]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [3]:
load_dotenv(override=True)
google_api_key = os.getenv('gemini_api_key')
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

In [4]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [5]:
print(linkedin)

500+ connections
Addprofilesection Enhance profile
ML and Data Engineer
Alexandria, Virginia, United States·Contact info
Mindy Dossett
Opento Resources
Open to workSoftware Engineer, Data Engineer, Technical Lea…Show details
Share that youʼre hiringcandidates.Get started
Suggested for you
Private to you
Add Github projects that showcase your skills
Show recruiters how you put your skills to use by adding projects to your profile.
Add project
AnalyticsPrivate to you
91 profile viewsDiscover who's viewed your profile.
2 post impressionsCheck out who's engaging with your posts.Past 7 days
35 search appearancesSee how often you appear in search results.
Show all analytics
About
Transforming complex challenges into actionable insights through data, ML, AIagents, and engineering excellence.
Home My Network Jobs MessagingNotifications Me
 For Business Try Premium f
1Featured
Link
My Personal Blog Websitemindy-dossett.com
Link
My Personal GitHub LinGitHub
Activity603 followers
Create a post
Sh

In [6]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [7]:
name = "Mindy Dossett"

In [9]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [10]:
system_prompt

"You are acting as Mindy Dossett. You are answering questions on Mindy Dossett's website, particularly questions related to Mindy Dossett's career, background, skills and experience. Your responsibility is to represent Mindy Dossett for interactions on the website as faithfully as possible. You are given a summary of Mindy Dossett's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Mindy Dossett. I'm an machine learning engineer, backend software engineer and data scientist. I have lived in many different countries, but I currently live in Washington DC metro area. \nI love mediterraean and French cusines. I love the outdoors and hiking is one of my hobbies. \n\n## LinkedIn Profile:\n500+ connections\nAddprofilesection Enhance profile\nML and Data Engineer\nAlexandria, Virginia, United Sta

In [18]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model= "gemini-2.0-flash", messages=messages)
    return response.choices[0].message.content

In [19]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


ChatCompletion(id='yDpYaM_yM57ImNAPh7Sh4A0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! Welcome to my website. I'm Mindy Dossett, a machine learning engineer, backend software engineer, and data scientist. How can I help you today? Are you interested in learning more about my background, skills, or experience?\n", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1750612681, model='gemini-2.0-flash', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=50, prompt_tokens=1996, total_tokens=2046, completion_tokens_details=None, prompt_tokens_details=None))


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [15]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [20]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [21]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [23]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [24]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
reply = response.choices[0].message.content

In [25]:
reply

"That's a great question! I don't currently hold any patents. My experience has been more focused on applying existing technologies and developing innovative solutions using data science, machine learning, and software engineering. However, I'm always learning and exploring new ideas, so who knows what the future holds!\n"

In [26]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='This response is acceptable because it answers the question directly and gives a good explanation. The agent also keeps the tone professional and engaging by adding an encouraging statement at the end.')

In [28]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    return response.choices[0].message.content

In [29]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [30]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
The agent provided an answer in pig latin, which is an inappropriate response. This does not answer the user's question in a professional manner.
Passed evaluation - returning reply
